In [6]:
#!/usr/bin/env python
# coding: utf-8

import os
import json
import pickle
import copy
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error, root_mean_squared_error
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from sklearn.preprocessing import normalize

# For SVD on rating matrix
from scipy.sparse import coo_matrix
from sklearn.decomposition import TruncatedSVD

# Fix for CUDA multiprocessing error:
import torch.multiprocessing as mp
try:
    mp.set_start_method('spawn', force=True)
except RuntimeError:
    pass

os.environ["TOKENIZERS_PARALLELISM"] = "false"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ---------------------------------------------------------------------
# Configuration & Hyperparameters
# ---------------------------------------------------------------------
DATA_PATH_INTERACTIONS = "./All_Beauty.jsonl"
DATA_PATH_ITEMS = "./meta_All_Beauty.jsonl"
PRETRAINED_TEXT_MODEL = "distilbert-base-uncased"

# The transformer model will fuse features into this hidden dimension.
HIDDEN_DIM = 512  
BATCH_SIZE = 2048
LR = 1e-4
EPOCHS = 15             # Increased epochs
NUM_FOLDS = 5

SVD_COMPONENTS = 2000      # Number of SVD components

LAMBDA_CLS = 1.5         # Classification loss weight
# Cache directories
os.makedirs("advanced_cache", exist_ok=True)
DATA_CACHE_PATH = "advanced_cache/data.pkl"
EMBED_CACHE_PATH = "advanced_cache/embeddings.pkl"
FEATURE_CACHE_PATH = "advanced_cache/features.pkl"

# ---------------------------------------------------------------------
# Data Loading and Caching with JSON Error Handling
# ---------------------------------------------------------------------
print("Loading cached data mappings...")
if os.path.exists(DATA_CACHE_PATH):
    with open(DATA_CACHE_PATH, 'rb') as f:
        data_cache = pickle.load(f)
    user2id = data_cache["user2id"]
    item2id = data_cache["item2id"]
    interactions_list = data_cache["interactions_list"]
    item_metadata = data_cache["item_metadata"]
else:
    user2id = {}
    item2id = {}
    interactions_list = []
    print("Loading Interactions...")
    i = 0
    with open(DATA_PATH_INTERACTIONS, 'r') as f:
        for line in tqdm(f, desc="Reading interactions", unit="line"):
            try:
                record = json.loads(line)
            except json.JSONDecodeError as e:
                print(f"Skipping line {i} due to JSONDecodeError: {e}")
                i += 1
                continue
            user_id = record.get("user_id")
            item_id = record.get("parent_asin")
            rating = record.get("rating", None)
            review_text = record.get("text", "")
            review_title = record.get("title", "")
            verified = record.get("verified_purchase", False)
            timestamp = record.get("timestamp", None)
            helpful_votes = record.get("helpful_votes", 0)
            if user_id not in user2id:
                user2id[user_id] = len(user2id)
            if item_id not in item2id:
                item2id[item_id] = len(item2id)
            interactions_list.append({
                "user_id": user2id[user_id],
                "item_id": item2id[item_id],
                "rating": float(rating) if rating is not None else 0.0,
                "review_text": review_text,
                "review_title": review_title,
                "verified": verified,
                "timestamp": timestamp,
                "helpful_votes": helpful_votes
            })
            i += 1
    print("Total Users:", len(user2id))
    print("Total Items:", len(item2id))
    print("Total Interactions:", len(interactions_list))
    print("Loading Item Metadata...")
    item_metadata = [None] * len(item2id)
    with open(DATA_PATH_ITEMS, 'r', encoding='utf-8') as f:
        for j, line in enumerate(tqdm(f, desc="Reading item metadata", unit="line")):
            try:
                record = json.loads(line)
            except json.JSONDecodeError as e:
                print(f"Skipping metadata line {j} due to JSONDecodeError: {e}")
                continue
            parent_asin = record.get("parent_asin")
            if parent_asin not in item2id:
                continue
            idx = item2id[parent_asin]
            title = record.get("title", "")
            avg_rating = record.get("average_rating", 0.0)
            rating_num = record.get("rating_number", 0.0)
            price = record.get("price", 0.0)
            item_metadata[idx] = {
                "title": title,
                "avg_rating": avg_rating,
                "rating_num": rating_num,
                "price": 0 if price is None else price
            }
    data_cache = {
        "user2id": user2id,
        "item2id": item2id,
        "interactions_list": interactions_list,
        "item_metadata": item_metadata
    }
    with open(DATA_CACHE_PATH, 'wb') as f:
        pickle.dump(data_cache, f)
    print("Data cached.")

# ---------------------------------------------------------------------
# Text Embeddings Using Pretrained Transformer (BERT)
# ---------------------------------------------------------------------
if os.path.exists(EMBED_CACHE_PATH):
    with open(EMBED_CACHE_PATH, 'rb') as f:
        embed_cache = pickle.load(f)
    user_text_embeds = embed_cache["user_text_embeds"]
    item_text_embeds = embed_cache["item_text_embeds"]
else:
    tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_TEXT_MODEL)
    text_model = AutoModel.from_pretrained(PRETRAINED_TEXT_MODEL).to(device)
    text_model.eval()
    @torch.no_grad()
    def encode_texts(texts, max_length=64):
        if len(texts) == 0:
            return torch.zeros(text_model.config.hidden_size)
        inputs = tokenizer(texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt').to(device)
        outputs = text_model(**inputs)
        cls_embeds = outputs.last_hidden_state[:, 0, :]
        return cls_embeds.mean(dim=0).cpu()
    print("Encoding Item Titles...")
    item_text_embeds = torch.zeros((len(item2id), text_model.config.hidden_size))
    for i in tqdm(range(len(item2id)), desc="Item title embeddings"):
        meta = item_metadata[i]
        if meta is None:
            emb = torch.zeros(text_model.config.hidden_size)
        else:
            emb = encode_texts([meta["title"]])
        item_text_embeds[i] = emb
    print("Aggregating User Review Embeddings...")
    user_reviews_map = {uid: [] for uid in range(len(user2id))}
    for inter in interactions_list:
        uid = inter["user_id"]
        user_reviews_map[uid].append(inter["review_text"])
    user_text_embeds = torch.zeros((len(user2id), text_model.config.hidden_size))
    for uid in tqdm(user_reviews_map.keys(), desc="User review embeddings"):
        revs = user_reviews_map[uid]
        sampled_revs = revs[:5]
        if sampled_revs:
            emb = encode_texts(sampled_revs)
        else:
            emb = torch.zeros(text_model.config.hidden_size)
        user_text_embeds[uid] = emb
    embed_cache = {
        "user_text_embeds": user_text_embeds,
        "item_text_embeds": item_text_embeds
    }
    with open(EMBED_CACHE_PATH, 'wb') as f:
        pickle.dump(embed_cache, f)
    print("Embeddings cached.")

# ---------------------------------------------------------------------
# Feature Engineering: Multi-Modal Item Features & User Features
# ---------------------------------------------------------------------
if os.path.exists(FEATURE_CACHE_PATH):
    with open(FEATURE_CACHE_PATH, 'rb') as f:
        feature_cache = pickle.load(f)
    user_features = feature_cache["user_features"]      # shape: (num_users, text_dim)
    item_text_features = feature_cache["item_text_features"]  # shape: (num_items, text_dim)
    item_numeric_features = feature_cache["item_numeric_features"]  # shape: (num_items, 3)
    item_svd_features = feature_cache["item_svd_features"]  # shape: (num_items, SVD_COMPONENTS)
else:
    for i, meta in enumerate(item_metadata):
        if meta is None:
            item_metadata[i] = {"avg_rating": 0.0, "rating_num": 0.0, "price": 0.0, "title":""}
    item_avg_ratings = torch.tensor([m["avg_rating"] for m in item_metadata], dtype=torch.float32)
    item_rating_counts = torch.tensor([m["rating_num"] for m in item_metadata], dtype=torch.float32)
    item_prices = torch.tensor([m["price"] for m in item_metadata], dtype=torch.float32)
    item_avg_ratings = (item_avg_ratings - item_avg_ratings.mean()) / (item_avg_ratings.std() + 1e-9)
    item_rating_counts = (item_rating_counts - item_rating_counts.mean()) / (item_rating_counts.std() + 1e-9)
    item_prices = (item_prices - item_prices.mean()) / (item_prices.std() + 1e-9)
    item_numeric_features = torch.cat([
        item_avg_ratings.unsqueeze(-1),
        item_rating_counts.unsqueeze(-1),
        item_prices.unsqueeze(-1)
    ], dim=-1)
    print("Computing SVD latent factors for items...")
    num_items = len(item2id)
    num_users = len(user2id)
    rows, cols, data_vals = [], [], []
    for inter in interactions_list:
        rows.append(inter["item_id"])
        cols.append(inter["user_id"])
        data_vals.append(inter["rating"])
    rating_matrix = coo_matrix((data_vals, (rows, cols)), shape=(num_items, num_users))
    svd = TruncatedSVD(n_components=SVD_COMPONENTS, random_state=42)
    svd_features = svd.fit_transform(rating_matrix)
    item_svd_features = torch.tensor(svd_features, dtype=torch.float32)
    print("SVD latent factors added. Dimension:", item_svd_features.size(1))
    item_text_features = item_text_embeds
    user_features = user_text_embeds
    feature_cache = {
        "user_features": user_features,
        "item_text_features": item_text_features,
        "item_numeric_features": item_numeric_features,
        "item_svd_features": item_svd_features
    }
    with open(FEATURE_CACHE_PATH, 'wb') as f:
        pickle.dump(feature_cache, f)
    print("Features cached.")

# ---------------------------------------------------------------------
# Create Fused Item Features by Concatenating Modalities
# ---------------------------------------------------------------------
# For each item, we concatenate text features, numeric features, and SVD features.
# The resulting item feature dimension is: text_dim + 3 + SVD_COMPONENTS.
text_dim = item_text_features.shape[1]
item_features = torch.cat([item_text_features, item_numeric_features, item_svd_features], dim=-1)

# ---------------------------------------------------------------------
# Define a PyTorch Dataset for Interactions
# ---------------------------------------------------------------------
class InteractionDataset(Dataset):
    def __init__(self, interactions, user_features, item_features):
        self.interactions = interactions
        self.user_features = user_features  # shape: (num_users, text_dim)
        self.item_features = item_features  # shape: (num_items, text_dim + 3 + SVD_COMPONENTS)
        
    def __len__(self):
        return len(self.interactions)
    
    def __getitem__(self, idx):
        inter = self.interactions[idx]
        user_id = inter["user_id"]
        item_id = inter["item_id"]
        rating = inter["rating"]
        user_feat = self.user_features[user_id]
        item_feat = self.item_features[item_id]
        # For classification, the target class is rating-1 (assuming ratings in {1,2,3,4,5})
        target_cls = int(rating) - 1 if rating >= 1 else 0
        return user_feat, item_feat, rating, target_cls

# ---------------------------------------------------------------------
# Transformer Fusion Recommender Model
# ---------------------------------------------------------------------
class TransformerFusionRecommender(nn.Module):
    def __init__(self, user_dim, item_dim, hidden_dim, num_layers=2, num_heads=4, num_classes=5):
        super().__init__()
        # Project input features to hidden_dim
        self.user_proj = nn.Linear(user_dim, hidden_dim)
        self.item_proj = nn.Linear(item_dim, hidden_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads, batch_first=True)
        self.user_transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.item_transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        # Cross-attention: user queries attend to item representation
        self.cross_attn = nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=num_heads, batch_first=True)
        self.fusion_linear = nn.Linear(2 * hidden_dim, hidden_dim)
        self.regressor = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
        self.classifier = nn.Linear(hidden_dim, num_classes)
        
    def forward(self, user_feat, item_feat):
        # user_feat: (batch, user_dim), item_feat: (batch, item_dim)
        user_h = self.user_proj(user_feat)   # (batch, hidden_dim)
        item_h = self.item_proj(item_feat)   # (batch, hidden_dim)
        # Add dummy sequence dimension
        user_h = user_h.unsqueeze(1)  # (batch, 1, hidden_dim)
        item_h = item_h.unsqueeze(1)  # (batch, 1, hidden_dim)
        user_enc = self.user_transformer(user_h)  # (batch, 1, hidden_dim)
        item_enc = self.item_transformer(item_h)  # (batch, 1, hidden_dim)
        # Cross-attention: use user_enc as query, item_enc as key/value
        cross_out, _ = self.cross_attn(query=user_enc, key=item_enc, value=item_enc)
        # Squeeze sequence dimension
        user_enc = user_enc.squeeze(1)
        cross_out = cross_out.squeeze(1)
        fused = self.fusion_linear(torch.cat([user_enc, cross_out], dim=-1))
        reg_out = self.regressor(fused).squeeze(-1)
        cls_logits = self.classifier(fused)
        return reg_out, cls_logits

# ---------------------------------------------------------------------
# Training and Evaluation Functions for the New Model
# ---------------------------------------------------------------------
def train_epoch_transformer(model, optimizer, dataloader, criterion_reg, criterion_cls):
    model.train()
    total_loss = 0.0
    for user_feat, item_feat, target_reg, target_cls in tqdm(dataloader, desc="Training epoch", unit="batch"):
        user_feat = user_feat.to(device)
        item_feat = item_feat.to(device)
        target_reg = target_reg.to(device)
        target_cls = target_cls.to(device)
        optimizer.zero_grad()
        reg_pred, cls_logits = model(user_feat, item_feat)
        loss_reg = criterion_reg(reg_pred, target_reg)
        loss_cls = criterion_cls(cls_logits, target_cls)
        loss = loss_reg + LAMBDA_CLS * loss_cls
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * user_feat.size(0)
    return total_loss / len(dataloader.dataset)

@torch.no_grad()
def evaluate_transformer(model, dataloader, criterion_reg, criterion_cls):
    model.eval()
    total_loss = 0.0
    preds = []
    targets = []
    for user_feat, item_feat, target_reg, target_cls in tqdm(dataloader, desc="Evaluating", unit="batch"):
        user_feat = user_feat.to(device)
        item_feat = item_feat.to(device)
        target_reg = target_reg.to(device)
        reg_pred, cls_logits = model(user_feat, item_feat)
        loss_reg = criterion_reg(reg_pred, target_reg)
        total_loss += loss_reg.item() * user_feat.size(0)
        preds.append(reg_pred.detach().cpu().numpy())
        targets.append(target_reg.detach().cpu().numpy())
    preds = np.concatenate(preds)
    targets = np.concatenate(targets)
    print(preds)
    print(targets)
    mae = mean_absolute_error(targets, preds)
    rmse = root_mean_squared_error(targets, preds)
    return mae, rmse, total_loss / len(dataloader.dataset)

# ---------------------------------------------------------------------
# Main Training Loop with Cross-Validation for the Transformer Model
# ---------------------------------------------------------------------
# Prepare complete item features and user features (precomputed)
user_feats = user_features  # shape: (num_users, text_dim)
# For items, use our concatenated features
item_feats = item_features  # shape: (num_items, text_dim + 3 + SVD_COMPONENTS)

# Create a list of interaction indices for cross-validation
all_indices = np.arange(len(interactions_list))
kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)
all_mae = []
all_rmse = []

criterion_reg = nn.SmoothL1Loss()  # Huber loss
criterion_cls = nn.CrossEntropyLoss()

for fold, (train_idx, test_idx) in enumerate(kf.split(all_indices)):
    print(f"Fold {fold+1}/{NUM_FOLDS}")
    train_inter = [interactions_list[i] for i in train_idx]
    test_inter = [interactions_list[i] for i in test_idx]
    
    train_dataset = InteractionDataset(train_inter, user_feats, item_feats)
    test_dataset = InteractionDataset(test_inter, user_feats, item_feats)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    # Define dimensions for the transformer model:
    user_dim = user_feats.shape[1]          # e.g. text_dim from BERT
    item_dim = item_feats.shape[1]          # text_dim + 3 + SVD_COMPONENTS
    model = TransformerFusionRecommender(user_dim, item_dim, hidden_dim=HIDDEN_DIM,
                                          num_layers=2, num_heads=4).to(device)
    optimizer = optim.Adam(model.parameters(), lr=LR)
    
    best_val_mae = float('inf')
    early_stop_patience = 5
    no_improve_epochs = 0
    
    for epoch in range(EPOCHS):
        print(f"Epoch {epoch+1}/{EPOCHS}")
        train_loss = train_epoch_transformer(model, optimizer, train_loader, criterion_reg, criterion_cls)
        val_mae, val_rmse, val_loss = evaluate_transformer(model, test_loader, criterion_reg, criterion_cls)
        print(f"Epoch {epoch+1}: Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val MAE:{val_mae:.4f} | Val RMSE: {val_rmse:.4f}")
        if val_mae < best_val_mae:
            best_val_mae = val_mae
            no_improve_epochs = 0
        else:
            no_improve_epochs += 1
            if no_improve_epochs >= early_stop_patience:
                print("Early stopping triggered.")
                break
    fold_mae, fold_rmse, _ = evaluate_transformer(model, test_loader, criterion_reg, criterion_cls)
    print(f"Fold {fold+1} Final: MAE: {fold_mae:.4f}, RMSE: {fold_rmse:.4f}")
    all_mae.append(fold_mae)
    all_rmse.append(fold_rmse)

print("Cross-Validation Results:")
print("MAE per fold:", all_mae)
print("RMSE per fold:", all_rmse)
print("Mean MAE:", np.nanmean(all_mae), "Mean RMSE:", np.nanmean(all_rmse))


/Users/ray.x.chen123/Documents/research/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading cached data mappings...
Loading Interactions...


Reading interactions: 701528line [00:02, 344268.15line/s]


Total Users: 631986
Total Items: 112565
Total Interactions: 701528
Loading Item Metadata...


Reading item metadata: 112590line [00:00, 167269.42line/s]


...

[3.6428125 4.907384  4.691933  ... 4.936069  1.0946246 3.8731773]
[5. 5. 5. ... 5. 1. 4.]
Epoch 5: Train Loss: 1.3669 | Val Loss: 0.3033 | Val MAE:0.5616 | Val RMSE: 0.9124
Epoch 6/15


Evaluating: 100%|██████████| 69/69 [00:03<00:00, 20.49batch/s]


[4.155861  5.1246815 4.9393806 ... 5.1910467 1.0759174 4.3416276]
[5. 5. 5. ... 5. 1. 4.]
Epoch 6: Train Loss: 1.3503 | Val Loss: 0.2886 | Val MAE:0.5232 | Val RMSE: 0.9073
Epoch 7/15


Training epoch:  80%|████████  | 220/275 [00:46<00:11,  4.65batch/s]

: 

In [ ]:
Cross-Validation Results:
MAE per fold: [0.49213757361534505, 0.5203920227670602, 0.5501907572041689, 0.5028691199894604, 0.5111886623057732]
RMSE per fold: [0.8844305602048712, 0.9108551409362662, 0.9460393990431739, 0.8631044222211626, 0.8369910783816911]
Mean MAE: 0.5153556271763615 Mean RMSE: 0.888284120157433